In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor

In [2]:
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')

In [3]:
submission = pd.read_csv('submission.csv', index_col=0)

In [4]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [5]:
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [6]:
data

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4,2020,3
24697788,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3,2020,3
24697789,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6,2020,3
24697790,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6,2020,3


In [7]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [8]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [9]:
train_num = df_num.sample(frac=1, random_state=0)
X = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
Y = train_num['CNT']
Z = train_num['CSTMR_CNT']

In [10]:
two_Y = np.zeros((len(Y),2))
two_Y[:,0] = Y
two_Y[:,1] = Z

In [11]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=X.columns)

In [12]:
import lightgbm as lgb
from tqdm import tqdm
y_pred =0
N =2
for i in tqdm(range(N)):
    model = MultiOutputRegressor(lgb.LGBMRegressor(random_state=i*101,max_depth= -1,
            learning_rate = 0.05,
            boosting_type='gbdt',
            objective= 'tweedie',
            metric= 'mse',
            num_boost_round=3000,
            #sigmoid=True,
            sub_row = 0.75,
            lambda_l2 = 0.1,nfold=5), n_jobs=-1)
    model.fit(X, two_Y)
    y_pred += model.predict(temp)
y_pred /= N

100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [59:40<00:00, 1790.09s/it]


In [13]:
temp['CNT']=np.clip(np.exp(y_pred[:,0])-1, 0, None)
temp['CSTMR_CNT'] = np.clip(np.exp(y_pred[:,1])-1, 0, None)

C:\Users\s_m04\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.
C:\Users\s_m04\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [24]:
train_num['gap']= train_num['CNT'] - train_num['CSTMR_CNT']
train_num.loc[train_num['gap'] <0,'mark'] = int32(0)
train_num.loc[train_num['gap'] ==0,'mark'] = int32(1)
train_num.loc[train_num['gap'] >0,'mark'] = int32(2)

NameError: name 'int32' is not defined

In [19]:
temp['gap']= temp['CNT'] - temp['CSTMR_CNT']
temp.loc[temp['gap'] <0,'mark'] = int32(0)
temp.loc[temp['gap'] ==0,'mark'] = int32(1)
temp.loc[temp['gap'] >0,'mark'] = int32(2)

In [20]:
X = train_num.drop(['AMT'],axis=1)
Y = np.log1p(train_num['AMT'])

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, Y, 
                                      test_size=0.2, random_state=0)
train_ds = lgb.Dataset(X_train, label=y_train)
val_ds = lgb.Dataset(X_test, label=y_test)
def rmsle_1(y_pred, data):
    y_true = np.array(data.get_label())
    score= np.sqrt(np.square(np.log1p(y_pred + 1) - np.log1p(y_true + 1)).mean())
    return 'rmsle', score, False
params = {
            'max_depth': -1,
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'tweedie_variance_power': 1.1,
            'metric': 'custom',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }
model = lgb.train(params,
                  train_ds,
                  5000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100,
                  feval=rmsle_1
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmsle: 0.0365593
[200]	valid_0's rmsle: 0.0332084
[300]	valid_0's rmsle: 0.0320631
[400]	valid_0's rmsle: 0.031436
[500]	valid_0's rmsle: 0.0309643
[600]	valid_0's rmsle: 0.0306253
[700]	valid_0's rmsle: 0.030333
[800]	valid_0's rmsle: 0.0300707
[900]	valid_0's rmsle: 0.0298594
[1000]	valid_0's rmsle: 0.0296756
[1100]	valid_0's rmsle: 0.0295272
[1200]	valid_0's rmsle: 0.0293802
[1300]	valid_0's rmsle: 0.0292413
[1400]	valid_0's rmsle: 0.029128
[1500]	valid_0's rmsle: 0.0290263
[1600]	valid_0's rmsle: 0.0289384
[1700]	valid_0's rmsle: 0.0288567
[1800]	valid_0's rmsle: 0.028772
[1900]	valid_0's rmsle: 0.0286985
[2000]	valid_0's rmsle: 0.0286178
[2100]	valid_0's rmsle: 0.0285208
[2200]	valid_0's rmsle: 0.0284472
[2300]	valid_0's rmsle: 0.0283838
[2400]	valid_0's rmsle: 0.0283351
[2500]	valid_0's rmsle: 0.0282856
[2600]	valid_0's rmsle: 0.0282313
[2700]	valid_0's rmsle: 0.0281812
[2800]	valid_0's rmsle: 0.028118


In [23]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CNT,CSTMR_CNT,gap,mark
0,0,0,0,1,1,1,2020,4,8846.934728,25584.050161,-16737.115433,0
1,0,0,0,1,1,1,2020,7,26668.251227,79782.511596,-53114.260369,0
2,0,0,0,1,1,2,2020,4,6.350924,18.607221,-12.256297,0
3,0,0,0,1,1,2,2020,7,13.839368,27.007073,-13.167705,0
4,0,0,0,1,1,3,2020,4,0.367920,0.946446,-0.578526,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7,0.006552,0.011533,-0.004981,0
1658856,16,30,14,0,2,4,2020,4,0.008416,0.018680,-0.010264,0
1658857,16,30,14,0,2,4,2020,7,0.010358,0.021676,-0.011318,0
1658858,16,30,14,0,2,5,2020,4,0.008421,0.018669,-0.010248,0


In [22]:
pred = model.predict(temp)
temp['AMT']= np.expm1(pred)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: mark